# SSP Spaces

This package includes SSPSpace (subtypes: RandomSSPSpace and HexgonalSSP), which defines an encoding to a high-dimensional Fourier space. RandomSSPSpace (with sampler='norm') is just defining Random Fourier Features, while HexgonalSSP define a structured, rather than random mapping. There is also a SPSpace for high-dimensional random encodings for discrete data. These classes all have methods for decoding, sampling, plotting, and binding (a method for composing embeddings). These are used by the SSP spaces.


In [25]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.insert(1, os.path.dirname(os.getcwd()))
os.chdir("..")
import vsagym

# Create SSP space
domain_dim = 2 # The dim of the 'x' space
bounds = np.tile([-1,1],(domain_dim,1)) # bounds of x space (needed only for decoding, can set as None if you don't need to decode)
ssp_type = 'hex'
if ssp_type=='hex':
    ssp_space = vsagym.spaces.HexagonalSSPSpace(domain_dim,
                     n_scales=6,n_rotates=6, # You can change the dim of the SSP either via the ssp_dim arg or (in the case of hex ssps) n_scales and n_rotates. Try changing these to see what happens
                     domain_bounds=bounds, length_scale=0.1, scale_min=1) 
elif ssp_type=='rand':
    ssp_space = vsagym.spaces.RandomSSPSpace(domain_dim,
                     ssp_dim=151, domain_bounds=bounds, length_scale=0.1)

# For HexSSPs, only certain dims are allowed. If you make the space with an invalid ssp_dim arg, it will just round ssp_dim to the closest 'ok' one, so you might need to check the ssp_dim of the returned ssp_space
d = ssp_space.ssp_dim 

# Some random x
x = np.array([0.1,-0.4])
phi = ssp_space.encode(x)

plt.figure()
ax = plt.subplot(111)
im = ssp_space.similarity_plot(phi, ax=ax)
plt.colorbar(im)
plt.title("Similarity/kernel plot: $k_{\\phi'}(x,y) = \\phi'\cdot \\phi(x,y)$ ")
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Let's try decoding
xhat = ssp_space.decode(phi, method='direct-optim') 
print(f"(x,y)={x}" )
print(f"SSP \phi(x,y)={phi}" )

print(f"Decoding error = {np.sqrt(np.sum((x-xhat)**2))}")

## SSP Gym Spaces


There are four gymnasium spaces included in this package. 
- SSPBox: SSPs encoding continuous data (i.e., from a gym.spaces.Box space). The underlying mapping (a SSPSpace object) will be automatically generated if not provided.
- SSPDiscrete: SPs encoding discete data (i.e., from a gym.spaces.Discrete space). The underlying mapping (a SPSpace object) will be automatically generated if not provided. (note that this is called SSPDiscrete rather than SPDiscrete to be consisent with the other spaces, this name may be changed in future versions of this package)
- SSPSequences: SSPs encoding seqewunces of continous or discrete data. Must be given a SSPBox or SSPDiscrete space
- SSPDict: A general purpose space for defining VSA/SSP style encoding and decoding schemes over multiple data types. This requries more set-up as shown in the examples later in this notebook 


In [26]:
# SSPBox

ssp_dim = 97
box_space = vsagym.spaces.SSPBox(-1, 1, 2, shape_out=ssp_dim, decoder_method='direct-optim', length_scale=0.1)
x = np.array([0.1, -0.3])
ssp = box_space.encode(x)


print(box_space.sample())
# box_space.samples(2)

In [21]:
# SSPDiscrete
discrete_space = vsagym.spaces.SSPDiscrete(3, shape_out=ssp_dim)
decoded_one = discrete_space.decode(discrete_space.encode(1))
print(discrete_space.sample())

In [15]:
decoded_one

In [27]:
seq_space = vsagym.spaces.SSPSequence(
        vsagym.spaces.SSPBox(-1, 1, 2, shape_out=ssp_dim, decoder_method='direct-optim', length_scale=0.1),
        length=3)
seq = np.array([[0.1, -0.3], [0, -0.1], [-0.2, 0.5]])
print(seq_space.decode(seq_space.encode(seq.reshape(-1))))
seq_space.sample()